### Title:

### Authors: 

### Last updated:

This notebook unpickles and examines the algorithm.

In [2]:
pip install -r "requirements.txt"

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt;'


In [27]:
import pickle 
import joblib as jl
import xgboost
import warnings
import numpy as np
import pandas as pd
warnings.filterwarnings("ignore", category=UserWarning)

Unpickle the model and examine its basic structure.

In [17]:
model=jl.load(open('wpi_model.pkl','rb'))
emb_model = model['model']

print('The model is a \n', type(model), '\n with keys given by: \n', model.keys())
print('\n \n The parameters of the model are given by: \n', emb_model.get_params())

The model is a 
 <class 'dict'> 
 with keys given by: 
 dict_keys(['model', 'performance', 'bootstrapped_performance', 'train_performance', 'feature_importance', 'flags', 'feature_selection_method'])

 
 The parameters of the model are given by: 
 {'memory': None, 'steps': [('prep', DataFrameMapper(df_out=True, drop_cols=[],
                features=[(['active_address_count'],
                           TransformerPipeline(steps=[('booltointtransformer',
                                                       BoolToIntTransformer()),
                                                      ('floattransformer',
                                                       FloatTransformer())]),
                           {}),
                          (['afspraken_no_show_count_last_year'],
                           TransformerPipeline(steps=[('booltointtransformer',
                                                       BoolToIntTransformer()),
                                                   

We can see the features within the `prep` step of the pipeline. Let's extract it.

In [20]:
prep_step = emb_model.named_steps['prep']
# Extract feature names
emb_features = []
for feature in prep_step.features:
    emb_features.extend(feature[0])

print("Feature names:", emb_features)

Feature names: ['active_address_count', 'afspraken_no_show_count_last_year', 'received_same_product_last_year', 'total_vermogen', 'days_since_last_dienst_end', 'applied_for_same_product_last_year', 'at_least_one_address_in_amsterdam', 'has_medebewoner', 'has_partner', 'avg_percentage_maatregel', 'deelnames_started_percentage_last_year', 'days_since_last_relocation', 'afspraken_no_contact_count_last_year', 'sum_inkomen_bruto']


We can also read the class labels of the final predictions and match them to the documentation.

In [31]:
# Access the final estimator (classifier) in the pipeline
final_estimator = emb_model.named_steps['clf'].best_estimator_

# Get the class labels
class_labels = final_estimator.classes_

print("The class labels are", class_labels, "which correspond to 'not onderzoekswaardig (doesn't merit further investigation)'and 'onderzoekswaardig (mertis further investigation)' respectively").

The class labels are [0 1] which correspond to 'not onderzoekswaardig (doesn't merit further investigation)'and 'onderzoekswaardig (mertis further investigation)' respectively


Create synthethic data corresponding to the features.

In [39]:
nrow = 10000
Random = np.random.randint(low=1, high=10000, size=(nrow, len(emb_features)))
df = pd.DataFrame(Random)
df.columns = emb_features
df.head()

,active_address_count,afspraken_no_show_count_last_year,received_same_product_last_year,total_vermogen,days_since_last_dienst_end,applied_for_same_product_last_year,at_least_one_address_in_amsterdam,has_medebewoner,has_partner,avg_percentage_maatregel,deelnames_started_percentage_last_year,days_since_last_relocation,afspraken_no_contact_count_last_year,sum_inkomen_bruto
0,8284,4253,7626,7025,6511,946,8293,4746,4091,2637,1112,5296,3226,2657
1,1561,695,8839,1008,6435,2840,9790,542,5247,4058,6539,71,9918,9966
2,45,1768,9923,3321,4136,2976,5757,8186,6417,3586,8219,8116,4849,2915
3,2154,1514,7773,6439,159,189,6283,6684,2505,7801,215,1659,2697,2090
4,4171,8716,6798,646,5078,6759,67,3662,3810,7115,4501,7972,1804,5856


In [42]:
emb_model.predict_proba(df)[0,:]

array([0.26189389, 0.73810611])